In [87]:
import numpy as np
import fitsio as fits
import glob
from tqdm import tqdm
from scipy.ndimage.filters import maximum_filter1d

In [167]:
indir = 'tess/16/'

In [168]:
files = glob.glob(indir + '*.fits')

### Time union

In [71]:
union_ts = set([])
for file in files:
    try:
        times = fits.read(file)['TIME']
    except:
        continue
    times = np.unique((np.round(times, 2)))
    times = set(times[~np.isnan(times)])
    union_ts.update(times)

In [72]:
len(union_ts)

2341

In [146]:
i2t = dict(enumerate(sorted(union_ts)))

In [148]:
t2i = {v:k for k,v in t2i.items()}

In [79]:
np.array([list(i2t.keys()), np.zeros(len(i2t))])

array([[1738.64, 1738.65, 1738.66, ..., 1763.3 , 1763.31, 1763.32],
       [   0.  ,    0.  ,    0.  , ...,    0.  ,    0.  ,    0.  ]])

### Process

In [159]:
outdir = 'tess/16_processed/'
os.makedirs(outdir, exist_ok=True)

In [99]:
window = 50

In [176]:
for file in tqdm(files):
    try:
        data = fits.read(file, ext=None)
    except:
        continue
    data = data.astype([(x, '<f8') for x in data.dtype.names])
    time = data['TIME']
    bool_time = ~np.isnan(time)
    sap_flux_o_err = data['SAP_FLUX']/data['SAP_FLUX_ERR']
    sap_flux_o_err = sap_flux_o_err/np.nanmax(sap_flux_o_err)
    bool_flux = ~np.isnan(sap_flux_o_err)
    valid = (bool_flux == bool_time) & (bool_time == True)
    time = time[valid]
    sap_flux_o_err = sap_flux_o_err[valid]
    assert(time.shape == sap_flux_o_err.shape)
    sap_flux_o_err = maximum_filter1d(sap_flux_o_err, size=window)
    res = np.stack([time, sap_flux_o_err])
    res[0] = np.round(res[0], 2)
    res = res[:,::window]
    empty = np.array([list(t2i.keys()), np.zeros(len(i2t))])
    ixs = np.array([t2i[x] for x in res[0]])
    empty[1][ixs] = res[1]
    outname = file.split(indir)[1].split('.')[0]
    outname = outdir + outname + '.npy'
    np.save(outname, empty)

100%|██████████| 2152/2152 [00:13<00:00, 158.55it/s]
